In [1]:
%cd ..

/Users/jason/repos/diffusion-motion-inbetweening


In [2]:
from pathlib import Path

from tqdm import tqdm
import numpy as np
import torch

from convert.joints2bvh import BVH, Quaternions, Animation
from convert.joints2bvh.joints2bvh import Joint2BVHConvertor, re_order, re_order_inv
from convert.joints2bvh.InverseKinematics import BasicInverseKinematics

from data_loaders.humanml.scripts.motion_process import recover_from_ric
from data_loaders.humanml.common.skeleton import Skeleton

/Users/jason/repos/diffusion-motion-inbetweening/convert/joints2bvh/InverseKinematics.py:2: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  import scipy.linalg as linalg


# `new_joint_vecs` - Unified Feature Vectors for Skeletons

- Shape (F, 263)

In [3]:
base_path = Path("dataset/HumanML3D")
data_path = base_path / "new_joint_vecs_abs_3d"
output_path = base_path / "bvh"
output_path.mkdir(exist_ok=True)

split_files = {
    "train": base_path / "train.txt",
    "val": base_path / "val.txt",
    "test": base_path / "test.txt"
}


In [4]:

split_ids = {}

for split, path in split_files.items():
    with open(path, "r") as f:
        split_ids[split] = f.read().splitlines()

{split: len(ids) for split, ids in split_ids.items()}


{'train': 23384, 'val': 1460, 'test': 4384}

In [20]:
converter = Joint2BVHConvertor()
n_convert = 16
iterations = 50

for split, ids in split_ids.items():

    if split in ["train", "val"]:
        continue

    split_dir = output_path / split
    split_dir.mkdir(exist_ok=True)

    for id in tqdm(ids[:n_convert]):
        # if id != "002246":
        #     continue

        npy_path = data_path / f"{id}.npy"
        text_path = base_path / "texts" / f"{id}.txt"

        with text_path.open("r") as f:
            first_text = f.readline().split("#")[0]
            first_text = first_text.replace(" ", "_").replace(":", "_").replace(".", "").replace(",", "")

        bvh_path = split_dir / f"{id}_fromvec{iterations}_{first_text}.bvh"

        sample = np.load(npy_path)
        print(sample.shape)

        joints = recover_from_ric(torch.from_numpy(sample), joints_num=22, abs_3d=True).numpy()
        print(joints.shape)

        bvh = converter.convert(joints, bvh_path, iterations=iterations, foot_ik=False)



  0%|          | 0/16 [00:00<?, ?it/s]

(199, 263)
(199, 22, 3)
Template exists: True


  6%|▋         | 1/16 [00:01<00:26,  1.78s/it]

(62, 263)
(62, 22, 3)
Template exists: True


 12%|█▎        | 2/16 [00:02<00:17,  1.22s/it]

...

 88%|████████▊ | 14/16 [00:17<00:02,  1.23s/it]

(144, 263)
(144, 22, 3)
Template exists: True


 94%|█████████▍| 15/16 [00:19<00:01,  1.24s/it]

(191, 263)
(191, 22, 3)
Template exists: True


100%|██████████| 16/16 [00:20<00:00,  1.31s/it]


## Interactive Plot of last file

In [6]:
import matplotlib.pyplot as plt
import ipywidgets

# from humanml/utils/paramUtil.py
t2m_kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

@ipywidgets.interact(frame=(0, joints.shape[0]-1))
def plot(frame=130):
    fig = plt.figure(figsize=(9, 9))
    plt.suptitle(f"{id}")

    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')

    ax.view_init(elev=10, azim=35, vertical_axis="y")
    ax.axis("equal")

    ax.set_ylim(0, 2)
    ax.set_xlim(-1, 1)
    ax.set_zlim(-1, 1)

    for chain in t2m_kinematic_chain:
        ax.plot3D(
            joints[frame, chain, 0],
            joints[frame, chain, 1],
            joints[frame, chain, 2],
        )


    plt.show()

interactive(children=(IntSlider(value=130, description='frame', max=190), Output()), _dom_classes=('widget-int…

---

## Lookup of the original data from KIT

002246.npy: `./pose_data/KIT/423/upstairs_downstairs01_poses.npy`

In [7]:
kit_npy_path = Path("../HumanML3D/pose_data/KIT/423/upstairs_downstairs01_poses.npy")
kit_npy_path.is_file()

True

In [8]:
kit_joints = np.load(kit_npy_path)

In [9]:
import matplotlib.pyplot as plt
import ipywidgets

# from humanml/utils/paramUtil.py
t2m_kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

@ipywidgets.interact(frame=(0, kit_joints.shape[0]-1))
def plot(frame=130):
    fig = plt.figure(figsize=(9, 9))
    plt.suptitle(f"{id}")

    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis')

    ax.view_init(elev=10, azim=35, vertical_axis="y")
    ax.axis("equal")

    ax.set_ylim(0, 2)
    ax.set_xlim(-1, 1)
    ax.set_zlim(-1, 1)

    for chain in t2m_kinematic_chain:
        ax.plot3D(
            kit_joints[frame, chain, 0],
            kit_joints[frame, chain, 1],
            kit_joints[frame, chain, 2],
        )


    plt.show()

interactive(children=(IntSlider(value=130, description='frame', max=191), Output()), _dom_classes=('widget-int…

---

## ~~TODO: Why does `TEST` produce a correct result while `fromvec` and `fromjoint` do not?~~
### It was `foot_ik`.. always set it to `False` for now.

---

In [18]:
n_convert = 16
iterations = 50


template = BVH.load("convert/joints2bvh/data/template.bvh", need_quater=True)


for split, ids in split_ids.items():

    if split in ["train", "val"]:
        continue

    split_dir = output_path / split
    split_dir.mkdir(exist_ok=True)

    for id in tqdm(ids[:n_convert]):
        if id != "002246":
            continue

        npy_path = data_path / f"{id}.npy"
        text_path = base_path / "texts" / f"{id}.txt"

        with text_path.open("r") as f:
            first_text = f.readline().split("#")[0]
            first_text = first_text.replace(" ", "_").replace(":", "_").replace(".", "").replace(",", "")

        bvh_path = split_dir / f"{id}_TEST2{iterations}_{first_text}.bvh"

        sample = np.load(npy_path)
        print(sample.shape)

        joints = recover_from_ric(torch.from_numpy(sample), joints_num=22, abs_3d=True).numpy()
        print(joints.shape)

        # ABOVE IS SAME AS BEFORE
        # EXPANDING THE CONVERT FUNCTION BELOW

        global_positions = joints[:, re_order]
        new_anim = template.copy()
        new_anim.rotations = Quaternions.Quaternions.id(global_positions.shape[:-1])
        new_anim.positions = new_anim.positions[0:1].repeat(global_positions.shape[0], axis=-0)
        new_anim.positions[:, 0] = global_positions[:, 0]  # For root global = local
        # new_anim.offsets = offsets[re_order]

        # Calculate quaternion rotations from global positions
        ik_solver = BasicInverseKinematics(new_anim, global_positions, iterations=iterations, silent=True)
        new_anim = ik_solver()

        BVH.save(bvh_path, new_anim, names=new_anim.names, frametime=1 / 20, order='zyx', quater=True)



  0%|          | 0/16 [00:00<?, ?it/s]

(191, 263)
(191, 22, 3)


100%|██████████| 16/16 [00:02<00:00,  7.74it/s]


---

## Other

In [11]:
t2m_raw_offsets = np.array(
    [
        [0, 0, 0],
        [1, 0, 0],
        [-1, 0, 0],
        [0, 1, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, 1, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [-1, 0, 0],
        [0, 0, 1],
        [0, -1, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, -1, 0],
        [0, -1, 0],
    ]
)
face_joint_indx = [2, 1, 17, 16]

In [12]:
parents = [0] * 22
parents[0] = -1

for chain in t2m_kinematic_chain:
    for j in range(1, len(chain)):
        parents[chain[j]] = chain[j-1]

parents

[-1, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 12, 13, 14, 16, 17, 18, 19]

In [13]:
skel = Skeleton(torch.from_numpy(t2m_raw_offsets), t2m_kinematic_chain, "cpu")

offset = skel.get_offsets_joints(torch.from_numpy(joints[0]))

offset

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.1031,  0.0000,  0.0000],
        [-0.1099,  0.0000,  0.0000],
        [ 0.0000,  0.1316,  0.0000],
        [ 0.0000, -0.3936,  0.0000],
        [ 0.0000, -0.3902,  0.0000],
        [ 0.0000,  0.1432,  0.0000],
        [ 0.0000, -0.4324,  0.0000],
        [ 0.0000, -0.4256,  0.0000],
        [ 0.0000,  0.0574,  0.0000],
        [ 0.0000,  0.0000,  0.1434],
        [ 0.0000,  0.0000,  0.1494],
        [ 0.0000,  0.2194,  0.0000],
        [ 0.1375,  0.0000,  0.0000],
        [-0.1434,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.1030],
        [ 0.0000, -0.1316,  0.0000],
        [ 0.0000, -0.1230,  0.0000],
        [ 0.0000, -0.2568,  0.0000],
        [ 0.0000, -0.2631,  0.0000],
        [ 0.0000, -0.2660,  0.0000],
        [ 0.0000, -0.2699,  0.0000]])

In [14]:
quats = skel.inverse_kinematics_np(joints, face_joint_indx)
quats.shape

/Users/jason/repos/diffusion-motion-inbetweening/data_loaders/humanml/common/quaternion.py:394: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1720538194305/work/aten/src/ATen/native/Cross.cpp:66.)
  v = torch.cross(v0, v1)


(191, 22, 4)

In [15]:
names = [
    "Hips",
    "LeftUpLeg",
    "LeftLeg",
    "LeftFoot",
    "LeftToe",
    "RightUpLeg",
    "RightLeg",
    "RightFoot",
    "RightToe",
    "Spine",
    "Spine1",
    "Spine2",
    "Neck",
    "Head",
    "LeftShoulder",
    "LeftArm",
    "LeftForeArm",
    "LeftHand",
    "RightShoulder",
    "RightArm",
    "RightForeArm",
    "RightHand",
]